# Laboratorio 2
##### - Juan José Osorio - 202021720
##### - Pablo Guatibonza - 202014393
##### - Juan Sebastián Hoyos - 201822167


### Carga de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## 1. Entendimiento y calidad de los datos
### 1.1 Carga de datos

In [5]:
df_original = pd.read_csv('data/MotorAlpes_data.csv')
df_motorAlpes = df_original.copy()
print(df_motorAlpes.shape)
df_motorAlpes.head(5)

(7115, 12)


,Unnamed: 0,year,km_driven,owner,seller_type,seats,fuel,transmission,mileage,engine,max_power,selling_price
0,0,NaN,145500.0,NaN,Individual,5.0,Diesel,Manual,23.40,1248.0,74.00,5451.90
1,1,2014.0,120000.0,Second Owner,Individual,5.0,Diesel,Manual,21.14,1498.0,103.52,4482.68
2,2,2006.0,140000.0,Third Owner,Individual,5.0,Petrol,Manual,17.70,13524.0,78.00,1914.22
3,3,2010.0,127000.0,First Owner,Individual,5.0,Diesel,Manual,23.00,NaN,NaN,2725.95
4,4,2007.0,120000.0,First Owner,Individual,5.0,Petrol,Manual,16.10,1298.0,88.20,1574.99


In [10]:
df_test = pd.read_csv('data/MotorAlpes_test.csv')
print(df_test.shape)
df_test.head(5)

(791, 11)


,Unnamed: 0,year,km_driven,owner,seller_type,seats,fuel,transmission,mileage,engine,max_power
0,3105,2014.0,52442.0,Second Owner,Trustmark Dealer,5.0,Petrol,Manual,18.50,1198.0,86.8
1,4030,2016.0,40000.0,First Owner,Individual,7.0,Petrol,Automatic,10.75,2694.0,163.7
2,1656,2014.0,200000.0,First Owner,Individual,5.0,Diesel,Manual,25.10,1498.0,98.6
3,3580,2016.0,20000.0,First Owner,Individual,5.0,Petrol,Manual,20.40,1197.0,81.8
4,5725,NaN,140000.0,NaN,Individual,5.0,Diesel,Manual,25.10,1498.0,98.6


### 1.2 Entendimiento de los datos

In [7]:
df_motorAlpes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7115 entries, 0 to 7114
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     7115 non-null   int64  
 1   year           6876 non-null   float64
 2   km_driven      6917 non-null   float64
 3   owner          6876 non-null   object 
 4   seller_type    7115 non-null   object 
 5   seats          7115 non-null   float64
 6   fuel           7115 non-null   object 
 7   transmission   7115 non-null   object 
 8   mileage        6917 non-null   float64
 9   engine         6835 non-null   float64
 10  max_power      6847 non-null   float64
 11  selling_price  6714 non-null   float64
dtypes: float64(7), int64(1), object(4)
memory usage: 667.2+ KB


In [8]:
df_motorAlpes.describe()

,Unnamed: 0,year,km_driven,seats,mileage,engine,max_power,selling_price
count,7115.000000,6876.000000,6.917000e+03,7115.000000,6917.000000,6835.000000,6847.000000,6714.000000
mean,4069.227829,2013.980948,6.911118e+04,5.411103,19.523473,1835.489539,141.981595,11261.208041
std,2347.371729,3.852565,5.796521e+04,0.953555,4.241574,2363.919253,274.956684,40765.694516
min,0.000000,1994.000000,1.000000e+00,2.000000,0.000000,4.000000,1.070000,1.910000
25%,2033.000000,2012.000000,3.400000e+04,5.000000,16.800000,1197.000000,68.050000,3210.560000
50%,4081.000000,2015.000000,6.000000e+04,5.000000,19.330000,1248.000000,83.100000,5451.900000
75%,6103.500000,2017.000000,9.400000e+04,5.000000,22.320000,1597.000000,104.680000,8480.740000
max,8127.000000,2020.000000,2.360457e+06,14.000000,46.816000,19972.000000,1995.640000,598983.440000


#### 1.2.1 Completitud

In [9]:
(df_motorAlpes.isnull().sum() / df_motorAlpes.shape[0]).sort_values(ascending = False)

selling_price    0.056360
engine           0.039353
max_power        0.037667
year             0.033591
owner            0.033591
km_driven        0.027829
mileage          0.027829
Unnamed: 0       0.000000
seller_type      0.000000
seats            0.000000
fuel             0.000000
transmission     0.000000
dtype: float64

#### 1.2.2 Duplicidad
##### En este caso los valores pueden estar repetidos ya que no se tiene un ID unico por lo que en teoría no se estaría violando ninguna regla en duplicidad de carros